Try it yourself!

In [1]:
# Imports
from tkinter import Tk, Canvas, mainloop, Label, Button
from pynput import mouse as m
from tensorflow.keras import models
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.layers import MaxPooling2D, Dense, Flatten, Dropout

In [2]:
# Class for the Drawing Panel and the Network
class DrawPanel:

    def __init__(self):

        # Define the Network

        num_filters = 8
        filter_size = 3
        pool_size = 2

        self.network = models.Sequential()
        self.network.add(layers.Conv2D(num_filters, filter_size, input_shape=(28, 28, 1), padding='same',
                                       activation='relu'))
        self.network.add(MaxPooling2D(pool_size=pool_size))
        self.network.add(layers.Conv2D(num_filters, filter_size, input_shape=(28, 28, 1), padding='same',
                                       activation='relu'))
        self.network.add(MaxPooling2D(pool_size=pool_size))
        self.network.add(Dropout(0.6))
        self.network.add(Flatten())
        self.network.add(Dense(256, activation='relu'))
        self.network.add(Dropout(0.6))
        self.network.add(Dense(128, activation='relu'))
        self.network.add(Dense(4, activation='softmax'))
        self.network.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'],)
        
        # Load the weights for the network
        self.network.load_weights('cnn.h5')

        # Variable declarations for drawing
        self.drawing = False
        self.oldX, self.oldY = -1, -1

        # Define the "backend" of the grid that is used for drawing
        self.grid = []
        for i in range(28):
            self.grid.append([])
        for k in range(28):
            for j in range(28):
                self.grid[k].append(0)

        # Instantiate the frame
        self.root = Tk()
        self.root.title('Smiley Recognition')
        self.root.geometry('280x360+0+0')

        # Instantiate the canvas and binds the event for the drawing
        self.canvas = Canvas(self.root, width=280, height=280, bg='white', cursor='dot')
        self.canvas.pack()
        self.canvas.bind("<Motion>", self.paint)

        # Instantiates the label with the detected chars
        self.label = Label(self.root, text="nothing detected yet")
        self.label.pack()

        # Clear button
        self.clear = Button(self.root, text='clear', command=self.clear)
        self.clear.pack()

        # Predict button
        self.predict_button = Button(self.root, text='predict', command=self.predict_drawing)
        self.predict_button.pack()

        # Instantiate the mouse listener
        self.mouseListener = m.Listener(on_click=self.on_click)
        self.mouseListener.start()

        # Mainloop for the frame
        mainloop()

    def paint(self, event):
        # Gets called when the user draws onto the canvas and ensures that only the squares of the 28x28 scheme are filled
        if self.drawing:
            x = event.x - event.x % 10
            y = event.y - event.y % 10
            self.canvas.create_rectangle(x, y, x + 20, y + 20, fill="black", outline="")
            try:
                self.grid[int(x / 10)][int(y / 10)] = 1
            except IndexError:
                pass

    def on_click(self, x, y, button, pressed):
        # Enables the drawing mechanic
        self.drawing = True
        if not pressed:
            self.drawing = False

    def clear(self):
        # Clear the canvas and reset the "backend" grid
        self.canvas.delete("all")
        self.label.config(text='')
        for k in range(28):
            for j in range(28):
                self.grid[k][j] = 0

    def predict_drawing(self):
        # Takes the image that has been drawn, transforms it and uses the network to predict the output
        image = np.array(self.grid).T
        image = image - 0.5
        image = np.expand_dims([image], axis=3)
        # print(image.reshape((28, 28, 1)).shape)
        prediction = self.network.predict(image)
        label = ['happy', 'sad', 'neutral', 'surprised']
        self.label.config(text=label[np.argmax(prediction, axis=1)[0]])

In [3]:
DrawPanel()